In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
train_full_df = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")
X = train_full_df.drop('SalePrice', axis = 1)
y = train_full_df.SalePrice
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 1)


In [2]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 921 to 1061
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   object 
 3   LotFrontage    959 non-null    float64
 4   LotArea        1168 non-null   int64  
 5   Street         1168 non-null   object 
 6   Alley          71 non-null     object 
 7   LotShape       1168 non-null   object 
 8   LandContour    1168 non-null   object 
 9   Utilities      1168 non-null   object 
 10  LotConfig      1168 non-null   object 
 11  LandSlope      1168 non-null   object 
 12  Neighborhood   1168 non-null   object 
 13  Condition1     1168 non-null   object 
 14  Condition2     1168 non-null   object 
 15  BldgType       1168 non-null   object 
 16  HouseStyle     1168 non-null   object 
 17  OverallQual    1168 non-null   int64  
 18  Overal

In [3]:
#object values filling with mode
object_cols = [cols for cols in X_train.columns if X_train[cols].dtype == "object"]
for i in object_cols:
    m = X_train[i].mode()[0]
    X_train[i].fillna(m, inplace = True)
    X_test[i].fillna(m, inplace = True)


object_cols_test = [cols for cols in X_test.columns if X_test[cols].dtype in ['object']]
for i in object_cols_test:
    m = X_test[i].mode()[0]
    X_test[i].fillna(m, inplace = True)
print(object_cols)
X_train.info()

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 921 to 1061
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1168 non-null   int64  
 1   MSSubClass     1168 non-null   int64  
 2   MSZoning       1168 non-null   object 
 3   LotFrontage    959 non-null    float64
 4   LotArea        1168 non-null   int64  
 5 

c:\users\my pc\appdata\local\programs\python\python37\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [4]:
#int type missing values handling
mis_col_int = [col for col in X_train.columns
              if X_train[col].dtype in ['int64', 'float64']]
for i in mis_col_int:
    m = X_train[i].mean()
    X_train[i].fillna(m, inplace = True )
    X_test[i].fillna(m, inplace = True)
    

In [5]:
'''now we have to check for cardinality of columns and then perform one hot encoding on columns with cardinality less than 10
  and drop columns with cardinality greater than 10.'''
low_car_cols = [col for col in X_train.columns if X_train[col].nunique()<10 and X_train[col].dtype == 'object']
high_car_cols = [item for item in object_cols if item not in low_car_cols]

X_train.drop(high_car_cols, axis = 1, inplace = True)
X_val.drop(high_car_cols, axis = 1, inplace = True)
X_test.drop(high_car_cols, axis = 1, inplace = True)

X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)
X_test = pd.get_dummies(X_test)
X_train, X_val = X_train.align(X_val, join = 'left', axis = 1)
X_train, X_test = X_train.align(X_test, join = 'left', axis=1)



c:\users\my pc\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
X_train.info()
col = [X_train.columns]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 921 to 1061
Columns: 231 entries, Id to SaleCondition_Partial
dtypes: float64(3), int64(34), uint8(194)
memory usage: 568.0 KB


In [11]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_val = std.transform(X_val)
X_test = std.fit_transform(X_test)

c:\users\my pc\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\users\my pc\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [14]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
model = XGBRegressor(n_estimators = 1000, learning_rate = 0.025)
model.fit(X_train, y_train,
         early_stopping_rounds = 5,
         eval_set = [(X_val, y_val)],
         verbose = False)
predictions = model.predict(X_val)
score = mean_absolute_error(predictions, y_val)
print(score)

15961.449191994863


In [16]:
y_pred = model.predict(X_test)
y_pred

array([131381.67 , 158716.73 , 190283.05 , ..., 162067.4  , 114777.445,
       234663.45 ], dtype=float32)

In [18]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('sample_submission.csv')
df = pd.concat([sub_df['Id'], pred], axis = 1)
df.columns = ['Id', 'SalePrice']
df.to_csv('sample_submission.csv', index = False)